In [1]:
from selenium import webdriver
from html.parser import HTMLParser
from time import sleep
from bs4 import BeautifulSoup

import random
import requests
import re

from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException

**С помощью selenium выгрузим все book_id всех авторов**

In [2]:
def random_sleep(offset=1.5, length=4):
    sleep(random.random() * length + offset)

author_id = {
  "Дарья Донцова":  29369,
  "Джеймс Роллинс": 29442,
  "Макс Фрай":      102994,
  "Эрин Хантер":    26149,
  "Дмитрий Емец":   35952
}

author_books = []

with webdriver.Chrome(ChromeDriverManager().install()) as driver:
    for name in author_id:
        driver.get("https://www.bookvoed.ru/author/books?id={author_id}".format(author_id=author_id[name]))
        random_sleep()
        last_height = driver.execute_script("return document.body.scrollHeight")
    
        #scroll down while not end of page
        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            random_sleep()

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                random_sleep()
                try:
                    driver.execute_script("window.scrollTo(0, {scr});".format(scr=driver.execute_script("return document.body.scrollHeight")-1000))
                    random_sleep()
                    elem = driver.find_element_by_xpath('//*[@id="author_books_list"]/div[12]/div[2]/a')
                except NoSuchElementException:
                    break
                elem.click()
                random_sleep()
            last_height = new_height

        #find all book id
        items = driver.find_elements_by_class_name("gf")
        
        #add books id to authors
        for item in items:
            classes = item.get_attribute('class').split()
            author_books.append(item.get_attribute("data-book"))
        
        print(len(author_books))


Checking for mac64 chromedriver:2.46 in cache
Driver found in /Users/koteuka/.wdm/chromedriver/2.46/mac64/chromedriver
954
1091
1368
1480
1774


**Функция для multiprocessing.pool, обрабатывает book_id и возвращает словарь book**

In [3]:
#fill the author_books_in_detail 
def multi_parsing(book_id):
    #Get the 'soup'
    book_url = "https://www.bookvoed.ru/book?id={id}".format(id=book_id)
    book_html = requests.get(book_url).text
    soup = BeautifulSoup(book_html, 'html.parser')

    #def RE for deleting \n and spaces at the beginning and at the end of the str
    def del_n_and_spaces(mystr):
        buf = re.sub("^\s+|\n|\r|\s+$", '', mystr)
        return buf if len(buf) > 0 else None

    #def for check correct of CSS object
    def check(obj):
        if obj is not None:
            if obj is list:
                if len(obj) > 0:
                    return obj
                else:
                    return None
            return obj
        return None

    #def for correct rating of book
    def check_rating(book):
        if len(re.findall("\d+\.\d+", book['style'])) != 0:
            return re.findall("\d+\.\d+", book['style'])[0]
        if len(re.findall("\d+", book['style'])) != 0:
            return re.findall("\d+", book['style'])[0]
        
    #Correct parse the table
    data = []
    table = soup.find('table', class_='tw')
    for row in table.findAll('tr'):
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([del_n_and_spaces(ele) for ele in cols if ele])

    #Get age rating
    age_rating = ''
    if soup.find('div', class_='ov nM') is not None:
        age_rating = "0+"
    elif soup.find('div', class_='pv nM') is not None:
        age_rating = "6+"
    elif soup.find('div', class_='qv nM') is not None:
        age_rating = "12+"
    elif soup.find('div', class_='rv nM') is not None:
        age_rating = "16+"
    elif soup.find('div', class_='sv nM') is not None:
        age_rating = "18+"

    #list to dict for correct None placement
    data = {re.sub("^\s+|:|\s+$", '', data[i][0]): data[i][1] for i in range(len(data))}

    #Please don't ask questions, it works -> everything is fine
    book = {
       "ID": book_id,
       "Название": del_n_and_spaces(soup.find('h1', attrs={'itemprop' : "name"}).contents[0]) if check(soup.find('h1', attrs={'itemprop' : "name"}))is not None else None,
       "Обложка": del_n_and_spaces(soup.find('a', class_='sf ee').img['src']) if check(soup.find('a', class_='sf ee')) is not None else None,
       "Возраст": age_rating,
       "Описание": del_n_and_spaces(soup.find('div', class_='lw').contents[0]) if check(soup.find('div', class_='lw')) is not None else None,
       "Рейтинг": check_rating(soup.find('div', class_='af')) if check(soup.find('div', class_='af')) is not None else None,
       "Понравилось": del_n_and_spaces(soup.find('a', class_='Ke Me ').text) if check(soup.find('a', class_='Ke Me ')) is not None else "0",
       "В закладки": del_n_and_spaces(soup.find('a', class_='Ke Le ff').text) if check(soup.find('a', class_='Ke Le ff')) is not None else "0",
       "Не понравилось": del_n_and_spaces(soup.find('a', class_='Ke Oe ').text) if check(soup.find('a', class_='Ke Oe ')) is not None else "0",
       "Цена": del_n_and_spaces(soup.find('div', class_='Hu Wu').text) if check(soup.find('div', class_='Hu Wu')) is not None else None,
       "Серия": data['Серия'] if 'Серия' in data else None,
       "Издательство": data['Издательство'] if 'Издательство' in data else None,
       "Год": data['Год'] if 'Год' in data else None,
       "Страниц": data['Страниц'] if 'Страниц' in data else None,
       "Переплёт": data['Переплёт'] if 'Переплёт' in data else None,
       "ISBN": data['ISBN'] if 'ISBN' in data else None,
       "Размеры": data['Размеры'] if 'Размеры' in data else None,
       "Формат": data['Формат'] if 'Формат' in data else None,
       "Код": data['Код'] if 'Код' in data else None,
       "В базе": data['В базе'] if 'В базе' in data else None,
       "Автор": soup.find('span', class_="Av").text,
       "Тематика": data['Тематика'] if 'Тематика' in data else None,
       "Тираж": data['Тираж'] if 'Тираж' in data else None
    }
    
    return book
            

**Распараллелим нахождение list of books**

In [4]:
from multiprocessing import Pool, Lock, Value
from time import sleep

mutex = Lock()
n_processed = Value('i', 0)

books = []

def func_wrapper(uid):
    res = multi_parsing(uid) 
    with mutex:
        global n_processed
        n_processed.value += 1
        if n_processed.value % 10 == 0:
            print(f"\r{n_processed.value} objects are processed...", end='', flush=True)
    return res

with Pool(processes=10) as pool:
    books = pool.map(func_wrapper, author_books)

1770 objects are processed...

In [5]:
import csv
import pandas as pd


template = {
   "ID": 6299459,
   "Название": "Макс Фрай(ХроникиЕхо)-1.Чуб земли",
   "Обложка": "/files/1836/64/31/10/7.jpeg",
   "Возраст": "16+",
   "Описание": "Трактир \"Кофейная гуща\" стоит на границе...",
   "Рейтинг": 98.7,
   "Понравилось": 910,
   "В закладки": 275,
   "Не понравилось": 12,
   "Цена": 405.0,
   "Серия": "Хроники Ехо",
   "Издательство": "АСТ",
   "Год": 2015,
   "Страниц": 320,
   "Переплёт": "твердый",
   "ISBN": "978-5-17-090543-0",
   "Размеры": "13,00 см x 20,00 см x 2,00 см",
   "Формат": "206.00mm x 130.00mm x 18.00mm",
   "Код": 1204921,
   "В базе": "Макс Фрай(ХроникиЕхо)-1.Чуб земли",
   "Автор": "Фрай Макс",
   "Тематика": "Отечественная",
   "Тираж": 5000
}

csv_file = "Names.csv"
try:
    with open(csv_file, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=template.keys())
        writer.writeheader()
        for i in range(200):
            writer.writerow(books[i])
except IOError:
    print("I/O error") 
    
ds = pd.read_csv(r"/Users/koteuka/Desktop/Names.csv", ',')
ds

FileNotFoundError: File b'/Users/koteuka/Desktop/Names.csv' does not exist